# Prokaryotic Promoters

This working prototype that reads a single string, verifies that it is a valid DNA sequence, finds exact matches to common promoter motifs, and reports an overall match score for each proposed promoter.

###  The DNA verification process

DNA normally consists of 4 different nucleotieds **A, T, G, and C.** We can scan the string to check if we find any string other than these characters. Linear Scanning can perform this operation in **O(n)** time.

### Promoter Matching Process

We can locate promoters using an exact matching algorithm. This process is very straightforward however is not always accurate. Fuzzy matching process can locate multiple possible promoter regions. Fuzzy matching technique generates a list of possible promoter regions with their corresponding probability of being a promoter.

##### Exact Matching

For exact promoter matching, Two regions indicating a promoter found together are the  **TTGACA (-35) and the TATAAT (-10)** regions. Which means, we can simply scan for these two regions to locate the promoter start regions. 

We can keep track of how spread apart **TTGACA(-35) and TATAAT (-10)** are, making sure their **distance (d)** $\le$ 21 and $\ge$ 15

To make things simple, for exact matching we will be looking for **TTGACA** and **TATAAT** or **TAXXXT** blocks.

To calculate the overall confidence score:
    
- We can use the normal distribution formula constrained between the range [21,15] where $\mu = 17$, and $\sigma = 0.8$.
- We calculate the (-35) or (-10) block probability using the corresponding hashtable (Dan's Research) for each block type we have. 

In [85]:
# Hashtable for -35 sequence
tf_hashtable = {
    "A" : [3,  10, 3, 58, 32, 54],
    "C" : [9,  3, 14, 13, 52,  5],
    "G" : [10, 5, 68, 10,  7, 17],
    "T" : [78, 82,15, 20, 10, 24],
}

# Hashtable for -10 sequence
t_hashtable = {
    "A" : [3, 89, 26, 59, 49, 3],
    "C" : [8, 3,  10, 12, 21, 5],
    "G" : [7, 1,  12, 15, 11, 2],
    "T" : [82, 7, 52, 14, 19, 89],
}

### Exact matching algorithm with hash table

If we consider a nucleotide $\tau$ at position $n$ in a DNA sequence $\Delta$ as the start of the -35 sequence

Given an example DNA Sequence ...TAGACAC...

Considering $\tau=$ **T** with position $n$ then we can generate a scoring system like below using the `tf_hashtable`:
| T | A | G | A | C | A | $\mu$ |
|---|---|---|---|---|---|-------|
|78 | 10| 68| 20|52 | 54| $\frac{47}{100}$   |

where $\mu$ is the score of the possible -35 sequence.



Once this step is done, we move forward i.e we go to $n + 1$ location where $\tau=$ A where we get another score for the nucleotide subset:

| A | G | A | C | A | C | $\mu$ |
|---|---|---|---|---|---|-------|
| 10| 68| 20| 52| 54| 5 | $\frac{-26}{100}$   |

Ideal way of calculating $\mu$ after the first subset is to subtract score at $n$ and add the score at $n + 1 + 5$ 

We store the list key=n value=[-35 sequence, score] ($B$) as a list. For the **-10** sequence we create a dictionary with [-10 sequence, n, score] ($A$).


In [86]:
# DNA_Sequence = "ATGCGTACGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC"
DNA_Sequence = """ATGCGTTGACAGTCCGATGCTTATAATCGGATCAGTGCTTGACACCGTAGCTTATAATGGCATCGTTGACATTGCCGATATAATCCGATGCGTTGACAGCTTGGATATAATCGTGGCATTGACACCGTATATAATGGCTAGCTTGACAACGTTGATATAATGCGTTCGATGCTTGACAGATCGCTTATAATGCGGATCCTTGACAGCTAGCTATAATGCCGTAGTTGACACTGATCTTATAATGGCATCGTTGACAGGCTAACGTATAATCGTGGATCTTGACAGCTAGTCTTATAATGCCGATCGTTGACACTTGGCTATAATCGTAGGCATCGTTGACAGATCGCTATAATGCCGTAGCTTGACACTTGGATATAAT"""


In [87]:
# Beta
tf_dict = {}

# Alpha
t_list = []

# Final List

flist = []

# calculating -35 scores
for n, T in enumerate(DNA_Sequence):
    # skipping non DNA characters
    # if n not in ['A', 'C', 'G', 'T']:
    #     continue
    if n + 6 <= len(DNA_Sequence):
        score = 0
        for j in range(0, 6):
            score += tf_hashtable[DNA_Sequence[n]][j]
        mu = (score) / 600
        tf_dict[n] = [DNA_Sequence[n:n + 6], mu]

In [88]:
tf_dict

{0: ['ATGCGT', 0.26666666666666666],
 1: ['TGCGTT', 0.38166666666666665],
 2: ['GCGTTG', 0.195],
 3: ['CGTTGA', 0.16],
 4: ['GTTGAC', 0.195],
 5: ['TTGACA', 0.38166666666666665],
 6: ['TGACAG', 0.38166666666666665],
 7: ['GACAGT', 0.195],
 8: ['ACAGTC', 0.26666666666666666],
 9: ['CAGTCC', 0.16],
 10: ['AGTCCG', 0.26666666666666666],
 11: ['GTCCGA', 0.195],
 12: ['TCCGAT', 0.38166666666666665],
 13: ['CCGATG', 0.16],
 14: ['CGATGC', 0.16],
 15: ['GATGCT', 0.195],
 16: ['ATGCTT', 0.26666666666666666],
 17: ['TGCTTA', 0.38166666666666665],
 18: ['GCTTAT', 0.195],
 19: ['CTTATA', 0.16],
 20: ['TTATAA', 0.38166666666666665],
 21: ['TATAAT', 0.38166666666666665],
 22: ['ATAATC', 0.26666666666666666],
 23: ['TAATCG', 0.38166666666666665],
 24: ['AATCGG', 0.26666666666666666],
 25: ['ATCGGA', 0.26666666666666666],
 26: ['TCGGAT', 0.38166666666666665],
 27: ['CGGATC', 0.16],
 28: ['GGATCA', 0.195],
 29: ['GATCAG', 0.195],
 30: ['ATCAGT', 0.26666666666666666],
 31: ['TCAGTG', 0.3816666666666666

In [89]:
# calculating -10 scores
for n, T in enumerate(DNA_Sequence):
    if n + 6 <= len(DNA_Sequence):
        score = 0
        for j in range(0, 6):
            score += t_hashtable[DNA_Sequence[n]][j]
        mu = (score) / 600
        t_list.append([DNA_Sequence[n:n + 6], n, mu])

In [90]:
t_list

[['ATGCGT', 0, 0.38166666666666665],
 ['TGCGTT', 1, 0.43833333333333335],
 ['GCGTTG', 2, 0.08],
 ['CGTTGA', 3, 0.09833333333333333],
 ['GTTGAC', 4, 0.08],
 ['TTGACA', 5, 0.43833333333333335],
 ['TGACAG', 6, 0.43833333333333335],
 ['GACAGT', 7, 0.08],
 ['ACAGTC', 8, 0.38166666666666665],
 ['CAGTCC', 9, 0.09833333333333333],
 ['AGTCCG', 10, 0.38166666666666665],
 ['GTCCGA', 11, 0.08],
 ['TCCGAT', 12, 0.43833333333333335],
 ['CCGATG', 13, 0.09833333333333333],
 ['CGATGC', 14, 0.09833333333333333],
 ['GATGCT', 15, 0.08],
 ['ATGCTT', 16, 0.38166666666666665],
 ['TGCTTA', 17, 0.43833333333333335],
 ['GCTTAT', 18, 0.08],
 ['CTTATA', 19, 0.09833333333333333],
 ['TTATAA', 20, 0.43833333333333335],
 ['TATAAT', 21, 0.43833333333333335],
 ['ATAATC', 22, 0.38166666666666665],
 ['TAATCG', 23, 0.43833333333333335],
 ['AATCGG', 24, 0.38166666666666665],
 ['ATCGGA', 25, 0.38166666666666665],
 ['TCGGAT', 26, 0.43833333333333335],
 ['CGGATC', 27, 0.09833333333333333],
 ['GGATCA', 28, 0.08],
 ['GATCAG', 2


**-10 and -35 distance**

Since the space between the promoter blocks have an effect in the overall probability. Once we get calculate $A$ and $B$, we iterate through $B$ 
in the following manner

for each element at index $i$ with value $\alpha \in A$:
    
where key $\gt \beta[1] - 12$ and $\lt \beta[1] - 22$ if it exists in $A$ 

if it does: we calculate final score as $S = \frac{((score_\alpha + score_\beta) + std_{-}normal(i - A[key]))}{3} $ 

append the resulting pair with the score and sort the result based on the final score




In [91]:
E = 2.718281828459045
PI = 3.141592653589793

def std_normal(mu, x, sigma):
    return 1/(sigma * (2 * PI)**0.5) * E**(-0.5 * ((x - mu)/sigma)**2)

In [92]:
# iterating through -10 sequence list

for i, n in enumerate(t_list):
    b_start = n[1] - 22
    b_end = n[1] - 12
    for k in range(b_start, b_end):
        if k in tf_dict.keys():
            final_score = ((n[2] + tf_dict[k][1]) + std_normal(17, n[1], 0.8)) / 3
            flist.append([tf_dict[k][0], n[0], k, n[1], final_score])
            # output format -35, -10, -35 index, -10 index, final score


# Output 

The output is formatted like this: -35 bp sequence, -10 bp sequence, -35 bp start index, -10 bp start index, final score percentage

In [93]:
flist.sort(key=lambda x: x[4], reverse=True)
flist

[['TGCGTT', 'TGCTTA', 1, 17, 0.439559283500597],
 ['ATGCGT', 'TGCTTA', 0, 17, 0.40122595016726365],
 ['GCGTTG', 'TGCTTA', 2, 17, 0.3773370612783747],
 ['GTTGAC', 'TGCTTA', 4, 17, 0.3773370612783747],
 ['CGTTGA', 'TGCTTA', 3, 17, 0.3656703946117081],
 ['TGCGTT', 'ATGCTT', 1, 16, 0.3305482300232036],
 ['ATGCGT', 'ATGCTT', 0, 16, 0.2922148966898703],
 ['TGCGTT', 'TTATAA', 1, 20, 0.27348024820098643],
 ['TTGACA', 'TTATAA', 5, 20, 0.27348024820098643],
 ['TGACAG', 'TTATAA', 6, 20, 0.27348024820098643],
 ['TGCGTT', 'TATAAT', 1, 21, 0.27333395279979783],
 ['TTGACA', 'TATAAT', 5, 21, 0.27333395279979783],
 ['TGACAG', 'TATAAT', 6, 21, 0.27333395279979783],
 ['TGCGTT', 'TAATCG', 1, 23, 0.2733333333334348],
 ['TTGACA', 'TAATCG', 5, 23, 0.2733333333334348],
 ['TGACAG', 'TAATCG', 6, 23, 0.2733333333334348],
 ['TTGACA', 'TCGGAT', 5, 26, 0.2733333333333334],
 ['TGACAG', 'TCGGAT', 6, 26, 0.2733333333333334],
 ['TCCGAT', 'TCGGAT', 12, 26, 0.2733333333333334],
 ['TCCGAT', 'TCAGTG', 12, 31, 0.27333333333

#### Displaying top 10 possible pairs

In [94]:
flist[:10]

[['TGCGTT', 'TGCTTA', 1, 17, 0.439559283500597],
 ['ATGCGT', 'TGCTTA', 0, 17, 0.40122595016726365],
 ['GCGTTG', 'TGCTTA', 2, 17, 0.3773370612783747],
 ['GTTGAC', 'TGCTTA', 4, 17, 0.3773370612783747],
 ['CGTTGA', 'TGCTTA', 3, 17, 0.3656703946117081],
 ['TGCGTT', 'ATGCTT', 1, 16, 0.3305482300232036],
 ['ATGCGT', 'ATGCTT', 0, 16, 0.2922148966898703],
 ['TGCGTT', 'TTATAA', 1, 20, 0.27348024820098643],
 ['TTGACA', 'TTATAA', 5, 20, 0.27348024820098643],
 ['TGACAG', 'TTATAA', 6, 20, 0.27348024820098643]]

## Preety Display TODO


##### Fuzzy Matching

These are the conditions to keep in mind: 
- In the TATAAT region, TAXXXT are conserved over 80% of the time (X means less conserved)

- In the TTGACA region of the first 3 (TTG) at least one is always conserved but 2 and most often 3 are usually conserved.

- Both units tend to occcur about 17 basepairs away from eachother, but this can vary from as little as 15 to as high as 21 (however 95% of the time it is betwren 16 and 18, with more than half of that being 17)

- The -35 region is further upstream from the -10 (-35 and -10 describe the base pair distance from the starting region)